In [29]:
import cv2 as cv
print("Current version of openCV=", cv.__version__)
import numpy as np
import os
import glob

Current version of openCV= 4.1.1


In [30]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
points_per_row = 6
points_per_column = 10
objp = np.zeros((points_per_column*points_per_row,3), np.float32)
objp[:,:2] = np.mgrid[0:points_per_row,0:points_per_column].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
#images = glob.glob('./calibration/pinhole/*.png')
images = glob.glob('./calibration/fisheye/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (points_per_row,points_per_column), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (points_per_row,points_per_column), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [31]:
flags = cv.CALIB_RATIONAL_MODEL
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], 
                                                  cameraMatrix=None, distCoeffs=None, flags=flags)
print("ret: ", ret)
print("K matrix: ", mtx)
print("D distortions: ", dist)
#print("rotation vecs: ", rvecs)
print("translation vecs: ", tvecs)


ret:  0.16471857306147508
K matrix:  [[774.81319236   0.         619.53074799]
 [  0.         774.97848948 362.39414286]
 [  0.           0.           1.        ]]
D distortions:  [[-9.83575767e-01  2.16092944e+00 -1.18618369e-02 -1.19677802e-03
  -2.30379818e+00 -9.38872359e-01  2.05546416e+00 -2.24187418e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
translation vecs:  [array([[-2.68652085],
       [ 0.34410923],
       [34.24107998]]), array([[ 4.36598231],
       [-1.15697129],
       [29.73191346]]), array([[-9.87430612],
       [-5.28368304],
       [16.80681039]]), array([[ 2.32905782],
       [ 0.40264431],
       [34.34506697]]), array([[-4.47337586],
       [-4.74745944],
       [16.13103069]]), array([[ 0.53724635],
       [-5.92307015],
       [19.81079552]]), array([[-0.97182916],
       [-3.0146015 ],
       [21.45750921]]), array([[-14.9647307 ],
       [  0.63950586],
       [ 22.40612329]]), array([[-11.5457236

In [32]:
img_dir = 'calibration/fisheye/'
#img_dir = 'fisheye_images/'
img_name  = '{}.png'
#dst_dir = 'calibration/pinhole_calibrated/'
dst_dir = 'calibration/fisheye_calibrated/'
for img_id in range(len(images)):
    img = cv.imread(img_dir + img_name.format(img_id))
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imshow("original: ", img)
    cv.imshow("undistort ",dst)
    cv.waitKey(500)
    cv.imwrite(dst_dir+img_name.format(img_id), dst)
cv.destroyAllWindows()

In [33]:
cv.destroyAllWindows()